In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('villamhub_rag_dataset.csv')
df.head(3)

,id,text,source
0,1,Villam Hub is an urban farming platform focuse...,VillamHub_Overview
1,2,The founder of Villam Hub is Wisdom Chibuzor. ...,VillamHub_Overview
2,3,Villam Hub was created by a team of young inno...,VillamHub_Overview


In [3]:
from sentence_transformers import SentenceTransformer

#  Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # gives 384-dimensional embeddings

In [4]:
from pinecone import Pinecone

# Connect to Pinecone using API key 
pc = Pinecone(api_key="pcsk_6BJeTV_8F1pjuyoyrb8FDK2QqrPKaosRuuHJrpUx52WwX44q7z6JmWqMuDb7R8SfJtSpzu")

# Connect to  existing index
index = pc.Index("villam-chatbot")


In [5]:
# Convert text to embeddings and prepare for Pinecone

vectors = []
for i, row in df.iterrows():   # goes through each row in the dataset 
    # convert text to embedding
    embedding = model.encode(row['text']).tolist()  
    
    # Create a Pinecone-friendly vector entry
    vectors.append({
        'id': f'id-{i}',
        'values': embedding,
        'metadata': {'text': row['text'], 'source': row['source']}
    })


# send data to pinecone
index.upsert(vectors=vectors, namespace="default")

C:\Users\Olamide\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 28 Jul 2025 13:26:50 GMT', 'Content-Type': 'application/json', 'Content-Length': '102', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '2255', 'x-pinecone-request-id': '1362463822965195472', 'x-envoy-upstream-service-time': '26', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 384 does not match the dimension of the index 768","details":[]}


In [ ]:

# Sample user query
query = "What is Villam Hub and what does it do?"

# Embed the query
query_embedding = model.encode(query).tolist()

# Search Pinecone for similar entries
response = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True,
    namespace="default"   # ← this is important
)

# Show the results
for match in response['matches']:
    print(f"Score: {match['score']}")
    print(f"Text: {match['metadata']['text']}\n")


In [ ]:
query = "Who is Villam Hub founder?"

# Embed the query
query_embedding = model.encode(query).tolist()

# Search Pinecone for similar entries
response = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True,
    namespace="default"   # ← this is important
)

# Show the results
for match in response['matches']:
    print(f"Score: {match['score']}")
    print(f"Text: {match['metadata']['text']}\n")